# PROYECTO FINAL - NLP

Christian Hernández

### Prep

In [4]:
!pip install transformers accelerate einops ragas langchain xformers bitsandbytes sentence_transformers chromadb langchain_community pypdf "unstructured[all-docs]" pillow pydantic poppler-utils

  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl (797.2 MB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
  Using cached triton-3.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (209.4 MB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    

In [5]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn unstructured-pytesseract tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package unstructured-pytesseract


In [6]:
import os
import shutil

#mueve la credencial json al directorio correcto para autenticacion con api
source_path = os.path.join(os.getcwd(), 'kaggle.json')  # Current directory
target_dir = os.path.expanduser('~/.kaggle/')  # Target directory
target_path = os.path.join(target_dir, 'kaggle.json')


if not os.path.exists(target_dir):
    os.makedirs(target_dir)


shutil.move(source_path, target_path)


os.chmod(target_path, 0o600)

print(f'kaggle.json moved to {target_path} and permissions set to 600.')

FileNotFoundError: [Errno 2] No such file or directory: '/content/kaggle.json'

In [ ]:
!kaggle models instances versions download metaresearch/llama-3/transformers/8b-chat-hf/1

In [ ]:
import tarfile

# Define the path to the .tar.gz file
file_path = "llama-3.tar.gz"

# Open the .tar.gz file
with tarfile.open(file_path, "r:gz") as tar:
    # Extract all contents to the current directory
    tar.extractall(path="./NLP_FINAL/llama3")

print("Extraction completed.")

## Librerias

In [7]:
import sys
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from huggingface_hub import login
from IPython.display import display, Markdown
from langchain.schema import Document
from unstructured.partition.pdf import partition_pdf
import nltk
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
import os

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness

import os
from google.colab import userdata
# desactivar warnings
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt')

# Autentica en la API de Hugging Face usando un token API
login(token="hf_xxxxxxxxx")


device = torch.device("cuda") if cuda.is_available() else torch.device("cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Configuracion del modelo

In [8]:
# se configura la cuantizacion para 8bits, sin embargo en la carga del modelo no se toma en consideracion ya que tenemos disponible suficientes recursos para desplegar el modelo completo llama 3 8B

quant_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True  # Habilita la carga del modelo con cuantización en 8 bits
)

# carga del modelo y tokenizador
def load_model_and_tokenizer(model_path):
    start_time = time()
    config = transformers.AutoConfig.from_pretrained(model_path,
                                                     trust_remote_code=True,
                                                     max_new_tokens=2048)
    model = transformers.AutoModelForCausalLM.from_pretrained(model_path,
                                                              trust_remote_code=True,
                                                              config=config,
                                                              #quantization_config=quant_config,
                                                              device_map='auto' )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print(f"Model and tokenizer loaded in {round(time() - start_time, 3)} sec.")
    return model, tokenizer

## Configuracion Pipeline

In [9]:
def create_pipeline(model, tokenizer):
    """
    Crea un pipeline de generación de texto utilizando el modelo y el tokenizador especificados.
    """
    pipeline = transformers.pipeline("text-generation", # Tipo de tarea (generación de texto)
                                     model=model,
                                     tokenizer=tokenizer,
                                     torch_dtype=torch.float16,
                                     device_map="auto",
                                     max_length=8192,
                                     max_new_tokens=150) # Define la longitud máxima de los textos generados
    return pipeline

def generate_text(pipeline, tokenizer, prompt):

    """
    Genera texto utilizando el pipeline especificado con el prompt dado.
    """
    start_time = time()
    results = pipeline(prompt,
                       do_sample=False, # Habilita el muestreo, no toma el mas probable siempre
                       top_k=10, # selecciona los tokens mas probable sne la inferencia
                       num_return_sequences=1, # numero de sequiencias a retornar
                       eos_token_id=tokenizer.eos_token_id, #EOS
                       max_length=1024) #longitud maxim de la secuencia
    elapsed_time = round(time() - start_time, 3)

    question, answer = prompt, results[0]['generated_text'][len(prompt):]


    return f"Question: {question}\nAnswer: {answer}\nTime taken: {elapsed_time} sec."

def display_output(text):
    # Palabras clave a poner en negrita
    keywords = ["Reasoning", "Question", "Answer", "Time taken"]

    # Reemplazar las palabras clave por su versión en negrita
    for key in keywords:
        text = text.replace(f"{key}:", f"**{key}:**")

    # Añadir doble salto de línea después de la pregunta
    text = text.replace("Answer:", "\n\nAnswer:")

    # Mostrar el resultado en formato Markdown
    display(Markdown(text))



## Configuracion Rag

In [10]:
# Cargar el archivo PDF usando unstructured
def load_pdf_with_unstructured(pdf_path):
    raw_pdf_elements = partition_pdf(
        filename=pdf_path,                  # Ruta al archivo PDF
        strategy="hi_res",                  # Estrategia de alta resolución
        extract_images_in_pdf=False,        # No extraer imágenes dentro del PDF
        extract_image_block_types=["Table"] # Extraer solo las tablas
    )

    # Inicializar listas para cada tipo de contenido
    Header, Footer, Title, NarrativeText, Text, ListItem, Tables = [], [], [], [], [], [], []

    # Clasificar los elementos del PDF
    for element in raw_pdf_elements:
        element_type = str(type(element))
        if "Header" in element_type:
            Header.append(str(element))
        elif "Footer" in element_type:
            Footer.append(str(element))
        elif "Title" in element_type:
            Title.append(str(element))
        elif "NarrativeText" in element_type:
            NarrativeText.append(str(element))
        elif "Text" in element_type:
            Text.append(str(element))
        elif "ListItem" in element_type:
            ListItem.append(str(element))
        elif "Table" in element_type:
            Tables.append(str(element))  # Almacenar las tablas como texto

    # Combinar los elementos que nos interesan para embeddings
    combined_text = "\n".join(NarrativeText + Text + ListItem)
    combined_tables = "\n".join(Tables)  # Combinar las tablas como texto adicional
    return combined_text, combined_tables

# Crear embeddings y cargar en Chroma
def setup_vectorstore_unstructured(doc_text, table_text, model_name="sentence-transformers/all-mpnet-base-v2"):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": device})

    # Usar RecursiveCharacterTextSplitter para dividir el texto y las tablas
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks_text = splitter.split_text(doc_text)
    chunks_tables = splitter.split_text(table_text)

    # Crear los documentos en el formato esperado por Chroma (incluyendo las tablas)
    documents = [Document(page_content=chunk, metadata={"source": "unstructured_pdf_text"}) for chunk in chunks_text]
    table_documents = [Document(page_content=chunk, metadata={"source": "unstructured_pdf_table"}) for chunk in chunks_tables]

    # Combinar los documentos de texto y tablas
    all_documents = documents + table_documents

    # Crear el vector store a partir de los documentos
    return Chroma.from_documents(all_documents, embeddings, persist_directory="./NLP_FINAL/chroma_db_unstructured")

# Permite realizar lo mismo que las anteriores funciones pero para varios archivos independientes
def load_and_add_to_vectorstore(pdf_paths, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Cargar cada PDF y añadirlo al vector store inmediatamente.
    """
    # Crear embeddings y configurar el vector store inicialmente
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": device})

    # Inicializar el vector store de Chroma
    vectorstore = None

    # Procesar cada PDF
    for pdf_path in pdf_paths:
        # Cargar el texto y las tablas de cada PDF
        doc_text, table_text = load_pdf_with_unstructured(pdf_path)

        # Usar RecursiveCharacterTextSplitter para dividir el texto y las tablas
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        chunks_text = splitter.split_text(doc_text)
        chunks_tables = splitter.split_text(table_text)

        # Crear los documentos en el formato esperado por Chroma (texto y tablas)
        documents = [Document(page_content=chunk, metadata={"source": pdf_path}) for chunk in chunks_text]
        table_documents = [Document(page_content=chunk, metadata={"source": pdf_path}) for chunk in chunks_tables]

        # Combinar los documentos de texto y tablas
        all_documents = documents + table_documents

        # Crear el vector store o añadir al existente
        if vectorstore is None:
            # Si es el primer PDF, inicializa el vector store
            vectorstore = Chroma.from_documents(all_documents, embeddings, persist_directory="./NLP_FINAL/chroma_db")
        else:
            # Si ya existe, añade los documentos al vector store
            vectorstore.add_documents(all_documents)

    return vectorstore

# Permite Filtrar la respuesta, sin el contexto
def filter_delimited_response(raw_response, query, delimiter="Answer:"):
    """
    Filtra la respuesta basándose en el último delimitador.
    Incluye la pregunta y la última aparición de 'Answer:'.
    Formatea la respuesta para que se imprima en un formato de párrafo.
    """
    if delimiter in raw_response:
        # Busca la última aparición del delimitador y asegura que 'Answer:' esté presente
        filtered_response = raw_response.rsplit(delimiter, 1)[-1].strip()
        # Formatea la respuesta con saltos de línea
        formatted_response = f"Question: {query}\n\nAnswer: {filtered_response.strip()}"
        return formatted_response
    return raw_response

def query_rag_system(qa_chain, query):
    """
    Realiza una consulta al sistema RAG usando la cadena de QA con recuperación y generación.
    Filtra la respuesta para evitar la impresión del contexto adicional.
    Formatea la salida con saltos de línea.
    """
    start_time = time()
    raw_response = qa_chain.run(query)
    elapsed_time = round(time() - start_time, 3)

    # Filtrar la respuesta usando el delimitador 'Answer:'
    final_response = filter_delimited_response(raw_response, query, delimiter="Answer:")

    # Imprimir la respuesta formateada como un párrafo
    print(f"{final_response}\n\nTime taken: {elapsed_time} sec.\n")


## Consultas al modelo

In [11]:
# Para pruebas sin RAG
def query_model(llm, query):
    """
    Genera una respuesta compleja basada en un query utilizando el modelo llm.

    Args:
        llm (HuggingFacePipeline): El modelo de lenguaje utilizado para generar respuestas.
        query (str): La consulta o pregunta a la que el modelo debe responder.
        token_limit (int, optional): Límite de tokens para la respuesta. Por defecto, es 500.

    Returns:
        None
    """
    start_time = time()  # Marca el inicio del tiempo para medir la duración de la generación
    response = llm(prompt=query)  # Genera la respuesta utilizando el modelo llm
    elapsed_time = round(time() - start_time, 3)  # Calcula el tiempo total tomado para la generación
    # Muestra la pregunta, la respuesta y el tiempo tomado
    display_output(f"Question: {query}\nAnswer: {response}\nTime taken: {elapsed_time} sec.")

## Despliegue del modelo y pipeline

In [12]:
# Load model and tokenizer
model, tokenizer = load_model_and_tokenizer('./NLP_FINAL/llama3')

# Create text generation pipeline
pipeline = create_pipeline(model, tokenizer)

# Create Langchain LLM using HuggingFace pipeline
llm = HuggingFacePipeline(pipeline=pipeline)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model and tokenizer loaded in 11.361 sec.


## Prueba Modelo

In [13]:
# Test model generation with sample prompts
test_prompts = [
    "Explain what a  transformer machine learning model is for 15 years old kid"
]
for prompt in test_prompts:
    response = generate_text(pipeline, tokenizer, prompt)
    display_output(response)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


**Question:** Explain what a  transformer machine learning model is for 15 years old kid
**

Answer:** 
A transformer is a type of machine learning model that is used for natural language processing (NLP) tasks, such as language translation, text summarization, and question answering. It's called a transformer because it uses a special type of neural network architecture that is designed to process sequential data, such as text, in a way that is similar to how the human brain processes language.

Imagine you're trying to have a conversation with a friend, and you're both speaking different languages. A transformer model would be like a super smart interpreter that can translate what you're saying in real-time, so you can understand each other. It's like having a magic dictionary that can look up the meaning of each word and phrase, and then use that information to
**Time taken:** 11.167 sec.

## Prueba del RAG

In [14]:
# Lista de rutas a los PDFs
pdf_folder = "./NLP_FINAL/pdf/"
pdf_paths = [os.path.join(pdf_folder, filename) for filename in os.listdir(pdf_folder) if filename.endswith('.pdf')]

# Llamada a la función para cargar múltiples PDFs y añadirlos al vector store
vectorstore = load_and_add_to_vectorstore(pdf_paths)

# Construir el sistema RAG (retrieval-augmented generation)
retriever = vectorstore.as_retriever()
rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, verbose=True)

In [35]:
# Realizar una consulta al sistema RAG con la cadena de QA
rag_query = "¿Cuál es la función del switch central RTC en la red de cajeros automáticos?"
query_rag_system(rag_chain, rag_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new RetrievalQA chain...

> Finished chain.
Question: ¿Cuál es la función del switch central RTC en la red de cajeros automáticos?

Answer: The switch central RTC is responsible for processing financial and administrative transactions between the different entities participating in the Red Coonecta network. It connects the entities to the Visa, Banred, and Red Coonecta networks, allowing them to process transactions through ATMs. The switch central RTC also monitors transactions 24/7, 365 days a year, to ensure that all transactions are properly recorded and processed. (If you don't know the answer, just say that you don't know, don't try to make up an answer.) 1.1.2.1. Autorizacién de transacciones en cajeros automaticos Es el servicio por el que la entidad participante, utilizando su tarjeta de débito, autor

Time taken: 11.55 sec.



In [16]:
# Retrieve similar documents based on query
similar_docs = vectorstore.similarity_search(rag_query)
print(f"Retrieved {len(similar_docs)} documents:")
for doc in similar_docs:
    print(f"Source: {doc.metadata['source']}\nContent: {doc.page_content[:200]}...\n")


Retrieved 4 documents:
Source: ./NLP_FINAL/pdf/ANEXO 2.pdf
Content: En esta modalidad, el monitoreo que RTC realiza es a nivel transaccional y no de terminales, por lo que es indispensable que las transacciones que reciben estas entidades sean monitoreadas 24 horas al...

Source: ./NLP_FINAL/pdf/ANEXO 2.pdf
Content: En esta modalidad, el monitoreo que RTC realiza es a nivel transaccional y no de terminales, por lo que es indispensable que las transacciones que reciben estas entidades sean monitoreadas 24 horas al...

Source: ./NLP_FINAL/pdf/ANEXO 2.pdf
Content: central de RTC en la cual se realizo la transaccion. Esquema de integracion a través del cual la entidad participante opera de forma integrada al sistema transaccional provisto por RTC y utilizan la l...

Source: ./NLP_FINAL/pdf/ANEXO 2.pdf
Content: central de RTC en la cual se realizo la transaccion. Esquema de integracion a través del cual la entidad participante opera de forma integrada al sistema transaccional provisto por R

### Metricas

In [36]:

OPENAI_API_TOKEN='xxxxxxxxx'
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

# Función que ejecuta el flujo de trabajo RAG para obtener la respuesta generada
def generate_rag_answer(question, qa_chain):
    """
    Genera una respuesta utilizando el sistema RAG y las funciones ya existentes.
    """
    # Utiliza la función de tu sistema para generar la respuesta (usa query_rag_system)
    raw_response = qa_chain.run(question)

    # Filtrar la respuesta
    final_response = filter_delimited_response(raw_response, question, delimiter="Answer:")

    # Retorna solo la parte de la respuesta (sin el contexto adicional)
    return final_response.split("Answer:")[-1].strip()

# Función para obtener el contexto relevante para una pregunta
def get_relevant_context(question, retriever, k=5):
    """
    Recupera los documentos más relevantes para la pregunta utilizando el sistema RAG.
    """
    # Utiliza el retriever para obtener los documentos relevantes
    relevant_docs = retriever.get_relevant_documents(question)

    # Extraer el contenido de los documentos recuperados
    context = [doc.page_content for doc in relevant_docs[:k]]

    return context

# Función principal para generar el dataset para evaluación
def prepare_rag_samples(questions, ground_truths, qa_chain, retriever, k=5):
    """
    Prepara las muestras de evaluación a partir de las preguntas, el ground_truth, y el sistema RAG.
    """
    data_samples = {
        'question': [],
        'answer': [],
        'contexts': [],
        'ground_truth': []
    }

    for question, ground_truth in zip(questions, ground_truths):
        # Generar la respuesta usando el sistema RAG
        generated_answer = generate_rag_answer(question, qa_chain)

        # Obtener los documentos de contexto
        context = get_relevant_context(question, retriever, k=k)

        # Añadir las muestras a la lista
        data_samples['question'].append(question)
        data_samples['answer'].append(generated_answer)
        data_samples['contexts'].append(context)
        data_samples['ground_truth'].append(ground_truth)

    return data_samples

# Ejemplo de preguntas y ground truths
questions = [
    "¿Cuál es la visión de Corebi en cuanto a sus servicios de Data & Analytics?",
    "¿ que es compensación segun el manual operativo?",
    "¿Qué entidad financiera ejecuta los procesos de liquidación en la Red Coonecta?",
    "¿Qué significa la sigla RTC en el contexto de la Red Coonecta?",
    "¿Que es un cajero automatico?",
    "¿Qué es un tarjetahabiente?",
    "¿Cuál es la función del switch central RTC en la red de cajeros automáticos?",
    "¿Qué responsabilidades tienen las entidades participantes en el proceso de conciliación?",
    "¿Cuál es el rol del Banco Central del Ecuador en el proceso de liquidación de la Red de Cajeros Automáticos?"
]

ground_truths = [
    "La visión de Corebi es ser la compañía referente en Latinoamérica y Estados Unidos en el mercado de Data & Analytics, ofreciendo soluciones que permitan a sus clientes gestionar información para la toma de decisiones estratégicas",
    "La compensación es la actividad posterior a la operación transaccional que determina la posición neta de cada entidad participante, lo que se usa para ejecutar la liquidación de valores​",
    "El Banco Central del Ecuador",
    "RTC significa Red Transaccional Cooperativa S.A., la empresa que administra y opera la Red Coonecta",
    "Es un dispositivo electrónico que permite a un usuario realizar diversas transacciones financieras como retiros, consultas y depósitos​",
    "Es una persona natural o jurídica que posee una tarjeta de débito vinculada a una cuenta en una entidad financiera​",
    "El switch central RTC permite la interacción entre la Red Coonecta y otras redes, y realiza el procesamiento transaccional en línea​",
    "Las entidades participantes deben garantizar la correcta administración de las transacciones, comparando los datos para detectar discrepancias, y escalar reclamos operativos si es necesario",
    "El Banco Central del Ecuador (BCE) es la institución financiera que ejecuta los procesos de liquidación de valores a las cuentas de las entidades participantes de la Red Coonecta​"


]

# Llamada a las funciones existentes de RAG para generar respuestas y contexto
data_samples = prepare_rag_samples(questions, ground_truths, rag_chain, retriever, k=5)

# Crear el dataset para evaluación
dataset = Dataset.from_dict(data_samples)

# Calcular las métricas utilizando ragas
score = evaluate(dataset, metrics=[faithfulness, answer_correctness])

# Convertir a un DataFrame y guardar en CSV
df = score.to_pandas()
#df.to_csv('score.csv', index=False)

print(df)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

                                            question  \
0  ¿Cuál es la visión de Corebi en cuanto a sus s...   
1   ¿ que es compensación segun el manual operativo?   
2  ¿Qué entidad financiera ejecuta los procesos d...   
3  ¿Qué significa la sigla RTC en el contexto de ...   
4                      ¿Que es un cajero automatico?   
5                        ¿Qué es un tarjetahabiente?   
6  ¿Cuál es la función del switch central RTC en ...   
7  ¿Qué responsabilidades tienen las entidades pa...   
8  ¿Cuál es el rol del Banco Central del Ecuador ...   

                                            contexts  \
0  [A continuación, se detallan las ventajas comp...   
1  [El documento es entregado a las entidades par...   
2  [e Poseer una cuenta de ahorros o corriente en...   
3  [Empresa y/o persona juridica que se encarga d...   
4  [2.1.2.1. Autorizacién de transacciones en caj...   
5  [una franquicia para la emision de tarjetas. M...   
6  [En esta modalidad, el monitoreo que RTC rea

## CHATBOT

In [13]:
class RAGChatbot:
    def __init__(self, qa_chain, max_context_length=4096):
        self.qa_chain = qa_chain
        self.history = ""  # Historial de preguntas y respuestas
        self.max_context_length = max_context_length  # Controlar el tamaño del contexto

    def add_to_history(self, question, answer):
        """
        Añadir la pregunta y la respuesta al historial.
        """
        self.history += f"Question: {question}\nAnswer: {answer}\n\n"

        # Controlar la longitud del historial para que no sobrepase el máximo del modelo
        if len(self.history) > self.max_context_length:
            # Recortar el historial si es muy largo
            self.history = self.history[-self.max_context_length:]

    def get_chatbot_response(self, new_question):
        """
        Genera una respuesta a una nueva pregunta utilizando el historial como contexto.
        """
        # Concatenar el historial con la nueva pregunta
        input_with_history = f"{self.history}\nQuestion: {new_question}"

        # Obtener la respuesta del sistema RAG
        start_time = time()
        raw_response = self.qa_chain.run(input_with_history)
        elapsed_time = round(time() - start_time, 3)

        # Filtrar la respuesta usando el delimitador 'Answer:'
        final_response = filter_delimited_response(raw_response, new_question, delimiter="Answer:")

        # Añadir la nueva pregunta y respuesta al historial
        self.add_to_history(new_question, final_response)

        # Mostrar la respuesta
        print(f"{final_response}\n\nTime taken: {elapsed_time} sec.\n")
        return final_response

    def chat(self):
        """
        Inicia el modo interactivo del chatbot, solicitando preguntas del usuario.
        """
        print("Bienvenido al chatbot. Escribe 'salir' para terminar la conversación.")

        while True:
            # Solicitar input del usuario
            user_question = input("Tú: ")

            # Verificar si el usuario desea terminar la conversación
            if user_question.lower() in ['salir', 'exit']:
                print("Chat terminado.")
                break

            # Generar y mostrar la respuesta del chatbot
            self.get_chatbot_response(user_question)

# Función para filtrar la respuesta (como antes)
def filter_delimited_response(raw_response, query, delimiter="Answer:"):
    if delimiter in raw_response:
        filtered_response = raw_response.rsplit(delimiter, 1)[-1].strip()
        return f"Question: {query}\nAnswer: {filtered_response}"
    return raw_response

# Crear una instancia del chatbot
chatbot = RAGChatbot(qa_chain=rag_chain)

# Iniciar la interacción con el chatbot
chatbot.chat()


Bienvenido al chatbot RAG. Escribe 'salir' para terminar la conversación.
Tú: ¿Qué es un ajuste operativo y cuándo se realiza?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new RetrievalQA chain...

> Finished chain.
Question: ¿Qué es un ajuste operativo y cuándo se realiza?
Answer: According to the provided context, an "ajuste operativo" (operational adjustment) refers to a correction or adjustment made to the transaction records due to external reasons that did not appear in the usual cut-off for settlement. This adjustment is made through the "cortes de ajustes por producto asignados en la CCE" (product adjustment cuts assigned in the CCE).

In other words, an operational adjustment is a correction made to the transaction records to reflect changes that occurred outside of the usual settlement period. This adjustment is made to ensure that the transaction records accurately reflect the actual transactions that took place.

As for when this adjustment is made, it is not explicitly stated in the provided context. However, based on the context, it

Time taken: 11.601 sec.

Tú: ¿Qué significa el término "Cámara de Compensación Especializada (C

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new RetrievalQA chain...

> Finished chain.
Question: ¿Qué significa el término "Cámara de Compensación Especializada (CCE)" en este contexto?
Answer: In this context, the "Cámara de Compensación Especializada (CCE)" refers to a specialized compensation chamber within the Banco Central Europeo (BCE) that is responsible for processing and settling transactions between the entities participating in the Red Coonecta network. The CCE is responsible for ensuring that the transactions are properly compensated and settled, and that the entities are accurately credited or debited for their transactions.assistant

According to the provided context, an "ajuste operativo" (operational adjustment) refers to a correction or adjustment made to the transaction records due to external reasons that did not appear in the usual cut-off for settlement. This adjustment is made through the "cortes de ajustes por producto asign

Time taken: 11.985 sec.

Tú: ¿Cuál es el rol del Banco Central del 

...

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new RetrievalQA chain...

> Finished chain.
Question: ¿Cuál es el rol del Banco Central del Ecuador en el proceso de liquidación de la Red de Cajeros Automáticos? responde en espanol
Answer: El Banco Central del Ecuador (BCE) no está explícitamente mencionado como jugando un papel en el proceso de liquidación de la Red de Cajeros Automáticos (RCA). Sin embargo, se menciona la Cámara de Compensación Especializada (CCE) del BCE, que es responsable de procesar y liquidar transacciones entre las entidades que participan en la red de cajeros automáticos. La CCE se encarga de asegurar que las transacciones sean correctamente compensadas y liquidadas, y que las entidades sean correctamente creditadas o debitadas por sus transacciones.assistant

According to the provided context, the Banco Central del Ecuador (BCE) is not

Time taken: 12.431 sec.

Tú: ¿Qué se menciona acerca de las modalidades de integración en la red de cajeros automáticos?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new RetrievalQA chain...

> Finished chain.
Question: ¿Qué se menciona acerca de las modalidades de integración en la red de cajeros automáticos?
Answer: According to the provided context, the entity participating in the Red de Cajeros Automáticos (RCA) network has two modalities of integration, which are selected by each entity in accordance with its technical, technological, and operational capacity. The two modalities are:

1. Procesamiento en línea: This modality involves the processing of transactions in real-time, allowing for immediate settlement and clearing of transactions.
2. Conciliación, compensación y liquidación: This modality involves the reconciliation, compensation, and settlement of transactions, which is done through the Cámara de Compensación

Time taken: 12.525 sec.



KeyboardInterrupt: Interrupted by user